
# Barbero durmiente

Para tratar el problema se  ha implementado una clase que simula la Barberria con los procesos que se ejecutan dentro de ella.
Se proporciona el esqueleto de la implementación de los procesos para que se marquen las zonas de exclusión mutua, de espera o de sincronazación. 
En el constructor de la Barberia se inicilizan todos los cerrojos, semaforos, condiciones y variables que se necesitan para la implementación.

In [2]:
import threading
import time
import random

class Barberia:
    def __init__(self):
        #mecanismos y variables de control clientes y acceso a la sala
        ####### inicializa cada uno con el tipo y valor oportuno####
        #####puedes cambiar los nombres o añadir nuevos#######

        self.MAX_CLIENTES = 3

        #controla la entrada en la sala de espera
        self.entrarEnSala = threading.Semaphore(3)
        #controla el número de clientes en la sala
        self.clientesEnSala = 0 
        #garantiza exclusion para modificacion de variables de control de sala o acceso a secciones
        self.exclusionMutuaSala = threading.Lock()
        
        #mecanismos y variables de contrrol del barbero
        ####### inicializa cada uno con el tipo y valor oportuno #######
        #####puedes cambiar los nombres o añadir nuevos#######
        #controla el acesso al sillon de corte, puede poner clientes en espera y ser despertados
        self.sillon = threading.Lock()
        #controla lel estado del barbero
        self.barberoOcupado = False
        #puede poner al barbero a dormir y ser despertado
        self.dormir = threading.Condition()
        

    def actividadBarbero(self):
        while True:
            ### INCLUYE EL CÓDIGO NECESARIO PARA PROTOCOLO INICIAL   ###
            
            with self.dormir:
                #print ("B1 ")
                ### consultar en exclusion mutua los clientes que hay en la sala###
                if self.clientesEnSala < 1:
                    #print ("B2 ")
                ### si no hay clientes esperando se pone a dormir hasta que le despierten ####
                    self.dormir.wait()
                #print ("B3 ")      
                print("El barbero da paso a un cliente")
                ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO ###
                with self.exclusionMutuaSala:
                    #self.clientesEnSala -= 1 
                    print("Clientes en sala: " + str(self.clientesEnSala))
                    self.barberoOcupado = True

                print("El barbero corta a un cliente")
                time.sleep(5)
                ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO Y AVISAR A LOS CLIENTES DE LA SALA ###
                with self.exclusionMutuaSala:
                    self.barberoOcupado = False


            
    def actividadCliente(self):

        print("El "+threading.current_thread().name+" llega a la puerta de la barbería.")
        ### INCLUYE EL CÓDIGO NECESARIO PARA PROTOCOLO INICIAL   ###
        
        
        with self.exclusionMutuaSala:
            #print ("C1 ")
            ### según estado entrar en la sala o no ####
            if self.clientesEnSala >= self.MAX_CLIENTES:
                # Si la sala está llena, el cliente se va sin esperar.
                print("La sala está llena, " + threading.current_thread().name + " se va sin pelarse.")
                return
            
            ### consultar en exclusion mutua el estado ddel barbero###
            if self.barberoOcupado or self.clientesEnSala > 0:
                #print ("C2 ")
                # Solo intentamos adquirir el semáforo si sabemos que debemos esperar.
                self.clientesEnSala += 1

            
        # Adquirir el semáforo fuera de la exclusión mutua para evitar deadlocks. (Aunque en realidad... para qué me vale? Si ya controlo las sillas con el número de clientes en sala...)
        self.entrarEnSala.acquire()
        print ("El " + threading.current_thread().name + " entra en la sala de espera.")

        with self.dormir:
            ### según estado entrar despertar al babero o no ####
            if not self.barberoOcupado:
                #print ("C4 ")
                self.dormir.notify()
        
        #print ("C5 ")
        self.sillon.acquire()
        print ("Está en el sillon el "+threading.current_thread().name)
        time.sleep(3)

        self.sillon.release()
        print ("El "+threading.current_thread().name+" abandona la barbería pelado. \n")
        
        with self.exclusionMutuaSala:
            self.clientesEnSala -= 1 
            print ("Clientes en sala: " + str(self.clientesEnSala))
        self.entrarEnSala.release()  # Libera el semáforo al salir de la sala de espera.
 
                


if __name__=="__main__":

        barberia = Barberia()
        
        ### INCLUYE EL CÓDIGO NECESARIO PARA CREAR E INICIALIZAR LOS HILOS  ###
       
        # Crear e iniciar el hilo del barbero
        barbero = threading.Thread(target = barberia.actividadBarbero)
        barbero.start()
        #barbero.join()
        
        ### clientes####
        # Crear e iniciar hilos para varios clientes
        for i in range(20):
            cliente = threading.Thread(target = barberia.actividadCliente, name=f'Cliente {i}')
            time.sleep(random.uniform(1, 5))  # Simula la llegada de clientes en momentos aleatorios
            cliente.start()
            #cliente.join()



El Cliente 0 llega a la puerta de la barbería.
El Cliente 0 entra en la sala de espera.
Está en el sillon el Cliente 0
El barbero da paso a un cliente
Clientes en sala: 0
El barbero corta a un cliente
El Cliente 1 llega a la puerta de la barbería.
El Cliente 1 entra en la sala de espera.
El Cliente 0 abandona la barbería pelado. 

Clientes en sala: 1
Está en el sillon el Cliente 1El barbero da paso a un cliente
Clientes en sala: 0
El barbero corta a un cliente

El Cliente 2 llega a la puerta de la barbería.
El Cliente 2 entra en la sala de espera.
El Cliente 3 llega a la puerta de la barbería.
El Cliente 3 entra en la sala de espera.
El Cliente 1 abandona la barbería pelado. 

Clientes en sala: 2
El barbero da paso a un cliente
Clientes en sala: 1
El barbero corta a un cliente
El Cliente 4 llega a la puerta de la barbería.
El Cliente 4 entra en la sala de espera.
El Cliente 5 llega a la puerta de la barbería.
El barbero da paso a un cliente
Clientes en sala: 3
El barbero corta a un cli

El Cliente 14 abandona la barbería pelado. 
Está en el sillon el Cliente 15

Clientes en sala: 3
El Cliente 18 entra en la sala de espera.
El barbero da paso a un cliente
Clientes en sala: 2
El barbero corta a un cliente
El Cliente 15 abandona la barbería pelado. 

Clientes en sala: 2
Está en el sillon el Cliente 18
El barbero da paso a un cliente
Clientes en sala: 1
El barbero corta a un cliente
El Cliente 18 abandona la barbería pelado. 
Está en el sillon el Cliente 17

Clientes en sala: 1
El Cliente 17 abandona la barbería pelado. 

Clientes en sala: 0
